In [100]:
import nltk
from numpy import array, ones, zeros, multiply
import numpy as np
import sys
import operator

UNK = "<unk>"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK
epsilon=1e-100
special = "<s>"
smoothing_trans = 1e-2
smoothing_trans_2 = 1e-5

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                    transition_proba_2 = None,smoothing_obs = 0.01):
            
            '''
            Debut taux erreur de 10% : (par pair)
            Avec hmm ordre 1 : 7%
            Avec hmm ordre 2 => 4%
            HMM ordre 3 => 3%'''
            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            #utile pour calcule la proba d un elelement en conaissant la classe observation_list
            self.state_liste_2 = []
            for e in state_list:
                for f in state_list:
                      self.state_liste_2.append((e,f))
            self.N2 = len(self.state_liste_2)
            print(str(self.N2) +" states 2")
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y_2 = self.state_liste_2
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba is None:
                self.transition_proba = zeros( (self.N2, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if transition_proba_2 is None:
                self.transition_proba2 = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba2=transition_proba_2
            print(self.transition_proba.shape)
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index_2 = {}
            self.Y_index = {}
            for i in range(self.N2):
                self.Y_index_2[self.omega_Y_2[i]] = i
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] =i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices

    
        def data2indices(self, sent): 
            """From one tagged sentence of the brown corpus: 
            - extract the words and tags 
            - returns two list of indices, one for each
            -> (wordids, tagids)
            """
            wordids = list()
            tagids  = list()
            for couple in sent:
                wrd = couple[0]
                tag = couple[1]
                if wrd in self.X_index:
                    wordids.append(self.X_index[wrd])
                else:
                    wordids.append(UNKid)
                tagids.append(self.Y_index[tag])
            return wordids,tagids
            
        def observation_estimation(self, pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill with counts
            for pair in pair_counts:
                wrd=pair[0]
                tag=pair[1]
                cpt=pair_counts[pair]
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i = self.Y_index[tag]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
            
        
        def transition_estimation(self, trans_counts):
            print(sum(trans_counts.values()))
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            self.indices_backs = {}
            # fill with counts
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index_2[pair[0]]
                self.transition_proba[j,i]=trans_counts[pair]
            # normalize
            self.transition_proba=self.transition_proba+smoothing_trans
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(1,self.N)
            #construit les indices a vister en fonction de l etat actuel
            for i in range(self.N2):
                nouveau = self.omega_Y_2[i]
                if not nouveau[1] in self.indices_backs:
                    self.indices_backs[nouveau[1]] = []
                self.indices_backs[nouveau[1]].append(i)
            print("indices_backs " , self.indices_backs['a'])
            
        #estime la proa 
        def transition_estimation_2(self, trans_counts):
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0]]
                self.transition_proba2[j,i]=trans_counts[pair]
            self.transition_proba2=self.transition_proba2+smoothing_trans_2
            self.transition_proba2=self.transition_proba2/self.transition_proba2.sum(axis=0).reshape(1,self.N)
            
        def init_estimation(self, init_counts):
            """Build the init. distribution"""
            # fill with counts
            for tag in init_counts:
                i=self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
             
        
        def supervised_training(self, pair_counts, trans_counts,init_counts,ctrans2,ctags):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.init_estimation(init_counts)
            self.transition_estimation_2(ctrans2) 
            #print ctags
            self.solo = zeros( (self.N,), float ) 
            for tags in ctags:
                self.solo[self.Y_index[tags]] = ctags[tags]
            self.solo=self.solo/sum(self.solo)
            print(self.solo)
                
        def get_trans(self,k,indi_2):
            #indi_2 : vers nouvel etat
            # k :ancien etat
            return  self.transition_proba[k,indi_2]
            '''index_old = self.Y_index[self.omega_Y_2[k][1]]
            if self.transition_proba[k,indi_2] > 0.0:
                return (1/3.0) * self.transition_proba[k,indi_2] + (1/3.0) * self.transition_proba2[index_old,indi_2]+  (1/3.0)  * self.solo[indi_2]
            index_old = self.Y_index[self.omega_Y_2[k][1]]
            return (1/2.0) *self.transition_proba2[index_old,indi_2] + (1/2.0) * self.solo[indi_2]'''
        
        def viverbit(self,mots):
            if(len(mots)==1):
                return [(mots[0],mots[0])]
            alpha = np.zeros((self.N2,len(mots)))
            xi = np.zeros((self.N2,len(mots)))
            #ajout unk
            #init
            #print(self.observation_estimation)
            #--------------- Modele ordre 0 ----------------------------------------------------------
            i = 0
            if mots[0] in self.X_index:
                i = self.X_index[mots[0]]
            #remplissage de la toute premiere colonne naivement
            #normalement c'est bon
            test = [self.initial_state_proba[self.Y_index[p2]] * self.observation_proba[i,self.Y_index[p2]] for p1,p2 in self.omega_Y_2]
            alpha[:,0] = test
            #remplissage de la seconde colonne avec un modele de markov d ordre 1
            #----------------Modele markov ordre 1 ---------------------------------------------------   
            for j in range(self.N2):
                i = 1
                indi = 0
                if mots[i] in self.X_index:
                    indi = self.X_index[mots[i]]
                indi_2 = 0
                indi_2 = self.Y_index[self.omega_Y_2[j][1]]
                
                indi_trans = self.Y_index[self.omega_Y_2[j][0]]
                maxs = 0
                indi_max = 0
                for k in self.indices_backs[self.omega_Y_2[j][0]]:
                    tmp = alpha[k,i-1] * self.observation_proba[indi,self.Y_index[self.omega_Y_2[j][1]]] * self.transition_proba2[indi_trans,indi_2] 
                    if tmp > maxs:
                        maxs = tmp
                        indi_max = k
                alpha[j,i] = maxs
                xi[j,i] = indi_max
            #---------------- Modele de markov d ordre 2 -----------------------------------------------
            for i in range(2,len(mots)):
                for j in range(self.N2):
                    indi = 0
                    if mots[i] in self.X_index:
                        indi = self.X_index[mots[i]]
                    indi_2 = 0
                    indi_2 = self.Y_index[self.omega_Y_2[j][1]]
                    maxs = 0
                    indi_max = 0
                    for k in self.indices_backs[self.omega_Y_2[j][0]]:
                        #self.transition_proba[k,indi_2]
                        tmp = alpha[k,i-1] * self.get_trans(k,indi_2)* self.observation_proba[indi,self.Y_index[self.omega_Y_2[j][1]]]
                        if tmp > maxs:
                            maxs = tmp
                            indi_max = k
                    alpha[j,i] = maxs
                    xi[j,i] = indi_max
            tags = []
            debut = len(xi)
            starting = np.argmax(alpha[:,len(xi[0])-1])
            #print(self.omega_Y_2[starting],xi[self.Y_index_2[self.omega_Y_2[starting]],len(xi[0])-1])
            #print("IMPRT ", self.omega_Y_2[int(xi[starting,len(xi[0])-1])])
            #tags.append(starting)
            #count = len(xi[0])
            '''while len(tags) != len(mots):
                count -= 1
                new_index = xi[starting,count]
                tags.append(new_index)
                starting = new_index'''
            tags = []
            debut = len(xi)
            starting = np.argmax(alpha[:,len(xi[0])-1])
            tags.append(starting)
            count = len(xi[0])
            while len(tags) != len(mots):
                count -= 1
                new_index = xi[starting,count]
                tags.append(new_index)
                #print(self.omega_Y_2[int(starting)])
                starting = new_index
            tags = tags[::-1]
            to_return  = []
            count = 0
            for i in tags:
                to_return.append((mots[count],self.Y_index_2.keys()[self.Y_index_2.values().index(i)][1]))
                count +=1
            return to_return
                 
        def evaluate(self,test_data):
            errors = 0
            total = 0
            erreur_false = 0
            total_false = 0
            erreur_2 = 0
            
            correction = 0
            correction_totale = 0
            
            correction_ajout = 0
            correction_ajout_totale = 0
            
            
            for i in range(len(test_data)):
                if i%50==0:
                    print(i)
                res = self.viverbit(map(operator.itemgetter(0), test_data[i]))
      
                
#                 print "Mot: ", test_data[i]
#                 print "Correction: ", res
#                 print 
                
                if sum([a!=b for a,b in test_data[i]])>0:
                    total_false +=1 
                    erreur_false += sum([a[1]!=b[1] for a,b in zip(res,test_data[i])])
                for a,b in zip(res,test_data[i]):
                    if(b[0] != b[1] and b[1]==a[1]):
                        correction +=1
                    if(b[1] == ''):
                        correction_ajout += 1 if b[1] == a[1] else 0
                        correction_ajout_totale += 1
                        
                    correction_totale +=1
                erreur_2 += sum([a!=b for a,b in test_data[i]])
                errors += sum([a[1]!=b[1] for a,b in zip(res,test_data[i])])
                total += len(res)
            print("Percentage of errors : " ,  (errors/float(total))*100.0)
            print("Pourcentage de correction  : ", (erreur_false/float(total_false)))
            #print("taux de correction d'ajout : ", ((correction_ajout/float(correction_ajout_totale))*100.0))
            print("Taux d erreur brut " , ((erreur_2/float(total))*100.0))
            print(total)
            print("taux correction calcul  2 " , correction, ((correction/float(correction_totale))*100.0),correction_totale)
      
            
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                  transition_proba_2 = None,
                 smoothing_obs = 0.01)
hmm.supervised_training(cpairs,ctrans,cinits,ctrans2,ctags)
print("init ")
print(cinits)
pprint("test")
'''for i in range(10):
    print(test[i])
    print(hmm.viverbit(map(operator.itemgetter(0), test[i])))'''

'''
ordre 2 basique
('Percentage of errors : ', 6.147540983606557)
('Pourcentage de correction  : ', 0.6606822262118492)
('Taux d erreur brut ', 10.1775956284153)
7320
('taux correction calcul  2 ', 485, 6.62568306010929, 7320)


sur 100 donnees:
('Percentage of errors : ', 7.578947368421053)
('Pourcentage de correction  : ', 0.8292682926829268)
('Taux d erreur brut ', 11.157894736842106)
'''

'''
('max : ', 0.0)
0
('max : ', 2.3713728451263007e-14)
0
('max : ', 1.8837501326215639e-13)
0
('max : ', 5.2683448452173056e-12)
0
('max : ', 1.7515599865523543e-10)
0
('max : ', 3.5099521454297783e-14)
0
('max : ', 5.4719708639882633e-15)
0
('max : ', 3.0561403772736174e-15)
0
('max : ', 1.0706123521905198e-16)
0
('max : ', 6.8508450216642249e-13)
0
'''

# hmm.evaluate(test)
#pprint(ctrans)
print(test[1])
# print(hmm.viverbit(map(operator.itemgetter(0), test[1])))

HMM creating with: 
676 states 2
26 states
26 observations
(676, 26)
85866
('indices_backs ', [0, 26, 52, 78, 104, 130, 156, 182, 208, 234, 260, 286, 312, 338, 364, 390, 416, 442, 468, 494, 520, 546, 572, 598, 624, 650])
[ 0.0737595   0.03358292  0.01445854  0.12636204  0.03171798  0.01911042
  0.02360164  0.07666518  0.04667943  0.00412103  0.00075436  0.02635365
  0.04482147  0.0833636   0.06829738  0.00104772  0.02247011  0.06818563
  0.05760365  0.02745725  0.09692808  0.01556912  0.01345971  0.02084963
  0.00191384  0.00086612]
init 
{'a': 3093, 'c': 1210, 'b': 1429, 'e': 880, 'd': 853, 'g': 490, 'f': 1035, 'i': 2352, 'h': 1048, 'k': 79, 'j': 54, 'm': 1171, 'l': 675, 'o': 2291, 'n': 757, 'q': 34, 'p': 1535, 's': 2172, 'r': 758, 'u': 288, 't': 5058, 'w': 1582, 'v': 136, 'y': 74, 'x': 3}
'test'
[('l', 'l'), ('e', 'e'), ('f', 'f'), ('t', 't'), ('i', 'i'), ('s', 's'), ('t', 't')]


In [ ]:
hmm.evaluate(test_modify)

0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:275: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400


In [76]:
print test_modify[79]

hmm.viverbit(map(operator.itemgetter(0), test_modify[79]))

[('t', 't'), ('h', 'h'), ('e', 'e'), ('z', 's'), ('e', 'e')]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:275: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[('t', 't'), ('h', 'h'), ('e', 'e'), ('z', 's'), ('e', 'e')]

# Compter les mots et les tags

In [1]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word,tag)
    * c_transitions: count of tag bigram 
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_transitions2 = dict()
    c_inits = dict()
    for sent in corpus:
        # we use i because of the transition counts
        for i in range(len(sent)):
            couple=sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            if i > 0:
                trans = (sent[i-1][1],tag)
                if trans in c_transitions2:
                    c_transitions2[trans]=c_transitions2[trans]+1
                else:
                    c_transitions2[trans]=1
            if i > 1:
                trans = ((sent[i-2][1],sent[i-1][1]),tag)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            if i==0:
                if tag in c_inits:
                    c_inits[tag]=c_inits[tag]+1
                else:
                    c_inits[tag]=1
                    
    return c_words,c_tags,c_pairs, c_transitions, c_inits, c_transitions2


# Création du vocabulaire (filtrage selon le nombre d'occurence)

In [2]:
def make_vocab(c_words, threshold):
    """ 
    return a vocabulary by thresholding word counts. 
    inputs: 
    * c_words : a dictionnary that maps word to its counts
    * threshold: count must be >= to the threshold to be included
    
    returns: 
    * a word list
    """
    voc = list()
    for w in c_words:
        if c_words[w] >= threshold:
            voc.append(w)
    return voc


# les données


# START

In [97]:
import cPickle
from pprint import pprint

with open("typos-data/test10.pkl", "rb") as input_file:
    test = cPickle.load(input_file)
    
with open("typos-data/train10.pkl", "rb") as input_file:
    train = cPickle.load(input_file)
print "Nombre de phrases de train = "+str(len(train))
print "Nombre de phrases de test  = "+str(len(test))

Nombre de phrases de train = 29057
Nombre de phrases de test  = 1501


In [98]:
cwords,ctags,cpairs,ctrans,cinits, ctrans2 = make_counts(train)
print "Nombre de mots  : "+str(len(cwords))
print "Nombre de tags  : "+str(len(ctags))
print "Nombre de paires: "+str(len(cpairs))
print "Nombre de trans : "+str(len(ctrans))+ " / "+ str(12*12)
print "Nombre de init. : "+str(len(cinits))
vocab = make_vocab(cwords,10)
print "Vocabulaire :"+str(len(vocab))

Nombre de mots  : 26
Nombre de tags  : 26
Nombre de paires: 127
Nombre de trans : 2489 / 144
Nombre de init. : 25
Vocabulaire :26


# Création du HMM

In [99]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None)

HMM creating with: 
676 states 2
26 states
26 observations
(676, 26)


# Apprentissage pas à pas 

In [6]:
hmm.observation_estimation(cpairs)
print hmm.observation_proba.sum(axis=0)


NameError: name 'hmm' is not defined

In [7]:
hmm.transition_estimation(ctrans)
print hmm.transition_proba.sum(axis=0)

NameError: name 'hmm' is not defined

In [8]:
hmm.init_estimation(cinits)
print sum(hmm.initial_state_proba)

NameError: name 'hmm' is not defined

# Apprentissage en une fois

In [9]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                 smoothing_obs = 0.001)
hmm.supervised_training(cpairs,ctrans,cinits)

NameError: name 'HMM' is not defined

# Modification données pour prendre en compte l'ajout de caractère

In [93]:
import numpy as np
import random
from copy import deepcopy

# On ajoute le caractère "" pour l'etiquette, pour signifiern que cette lettre n'était pas sensé être là

# On ajoute un caractère a 10% des données d'entrainement et a 10% des données de test
def add_caracter(data, letters):
    nb_word_train = len(data)
    nb_modif = int(np.floor(0.1 * nb_word_train))
    word_to_modify = random.sample(range(0, nb_word_train), nb_modif)

    data_modify = deepcopy(data)
    for i in word_to_modify:
        word = data_modify[i]
        indice_add = random.randint(0, len(word))
        letter = random.choice(letters)
        if indice_add != len(word):
            word = word[:indice_add] + [(letter,'')] + word[indice_add:]
        else:
            word = word[:indice_add] + [(letter,'')]
        data_modify[i] = word
    return data_modify




def remove_caracter(data):
    nb_word_train = len(data)
    nb_modif = int(np.floor(0.1 * nb_word_train))
    word_to_modify = random.sample(range(0, nb_word_train), nb_modif)
    data_modify = deepcopy(data)
    for i in word_to_modify:
        word = data_modify[i]
        indice_remove = random.randint(0, len(word)-1)
        if indice_remove <= len(word)-1 and indice_remove != 0:
            word = word[0:indice_remove] + [('',word[indice_remove][1])] + word[indice_remove+1:]
        elif indice_remove == len(word)-1 :
            word = word[:indice_remove-1] + [('',word[indice_remove][1])]
        else:
            word = [('',word[indice_remove][1])] + word[indice_remove+1:]
        data_modify[i] = word
    return data_modify


def remove_caracter2(data):
    nb_word_train = len(data)
    nb_modif = int(np.floor(0.1 * nb_word_train))
    word_to_modify = random.sample(range(0, nb_word_train), nb_modif)
    data_modify = deepcopy(data)
    for i in word_to_modify:
        word = data_modify[i]
   
        indice_remove = random.randint(0, len(word)-1)
        
        if indice_remove < len(word)-1 :
            word2 = word[:indice_remove] #+ [('',word[indice_remove][1])] + word[indice_remove+1:]
            #faire une boucle de decalage
            i = indice_remove
            while i < len(word)-1: #s'arrêter à l'avant dernier indice
                word2 = word2 + [(word[i+1][1],word[i][1])]
                i = i + 1
            word2 = word2 + [("",word[i][1])]
        
        else:   
            word2 = word[:indice_remove] + [('',word[indice_remove][1])]
     
        data_modify[i] = word2
            
    return data_modify




In [94]:
#train_modify = add_caracter(train, list(vocab))
#test_modify = add_caracter(test, list(vocab))

train_modify = remove_caracter2(train)
test_modify = remove_caracter2(test)

In [56]:
print train_modify[2]
print test[2]

[('a', 'a'), ('b', 'b'), ('o', 'o'), ('u', 'u'), ('t', 't')]
[('i', 'i'), ('s', 's')]


In [57]:
cwords,ctags,cpairs,ctrans,cinits, ctrans2 = make_counts(train_modify)
print "Nombre de mots  : "+str(len(cwords))
print "Nombre de tags  : "+str(len(ctags))
print "Nombre de paires: "+str(len(cpairs))
print "Nombre de trans : "+str(len(ctrans))+ " / "+ str(12*12)
print "Nombre de init. : "+str(len(cinits))
vocab = make_vocab(cwords,10)
print "Vocabulaire :"+str(len(vocab))

Nombre de mots  : 27
Nombre de tags  : 26
Nombre de paires: 148
Nombre de trans : 2565 / 144
Nombre de init. : 25
Vocabulaire :27


In [46]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None)

HMM creating with: 
676 states 2
26 states
27 observations
(676, 26)


In [34]:

'''
(u'Some', u'DET'), (u'ten', u'NUM'), (u'years', u'NOUN'), (u'ago', u'ADV'), (u'that', u'DET'),
(u'page', u'NOUN'), (u'was', u'VERB'), (u'torn', u'VERB'), (u'out', u'PRT'), (u',', u'.'), 
(u'I', u'PRON'), (u"don't", u'VERB'), (u'know', u'VERB'), (u'by', u'ADP'), (u'whom', u'PRON'), (u'.', u'.')]
'''


'''
Projet : variable n est plus mot mais caractere
observer un mot => sequence d etats la version correcte
Xik = convluwion
Yik = conclusion

pour que ce soit plus dur => modele de second ordre

donnees (bon char, mauvais char)

 deux version : 10 : 10% d erreurs
                 20 : 20%  d erreurs
                 
3 points : hmm et biterdi : taux d erreurs du modele

modele markov ordre deux : => proba  d un etat depend des deux etats precedents P(Yt|Yt-1,Yt2)

dans viberti : on propage (dans delta) la proba du meilleur chemin pour arriver a un instant t dans un etat donne
=> quand rempli t on peut remplir t+1 (par hypothese) markov d ordre 1 => donc a ordre 2 il faut faire changement de varibale
besoin de garder la trace de tout ce qui est contionnement (Yt garde trace de ce qui est en t-1 et t-2 )
taille devient s² => delta de taille (K*N^2)

3 novembre : finir partie 1
'''

'\nProjet : variable n est plus mot mais caractere\nobserver un mot => sequence d etats la version correcte\nXik = convluwion\nYik = conclusion\n\npour que ce soit plus dur => modele de second ordre\n\ndonnees (bon char, mauvais char)\n\n deux version : 10 : 10% d erreurs\n                 20 : 20%  d erreurs\n                 \n3 points : hmm et biterdi : taux d erreurs du modele\n\nmodele markov ordre deux : => proba  d un etat depend des deux etats precedents P(Yt|Yt-1,Yt2)\n\ndans viberti : on propage (dans delta) la proba du meilleur chemin pour arriver a un instant t dans un etat donne\n=> quand rempli t on peut remplir t+1 (par hypothese) markov d ordre 1 => donc a ordre 2 il faut faire changement de varibale\nbesoin de garder la trace de tout ce qui est contionnement (Yt garde trace de ce qui est en t-1 et t-2 )\ntaille devient s\xc2\xb2 => delta de taille (K*N^2)\n\n3 novembre : finir partie 1\n'